In [1]:
import os
import sys


def report_python_env_info():
    # Where are we !?
    print(f"Current working directory (cwd):\n  {os.getcwd()}")
    print(f"Python version:\n  {sys.version}")
    print(f"Full path to python executable:\n  {sys.executable}")


report_python_env_info()


Current working directory (cwd):
  /home/brandon/Omdena/Omdena HeartKinetics/HeartKinetics - Project
Python version:
  3.9.16 (main, Jan 11 2023, 16:05:54) 
[GCC 11.2.0]
Full path to python executable:
  /home/brandon/anaconda3/bin/python


In [2]:
import xarray as xr
import json
from typing import Union
from pathlib import Path
import glob


def to_path(p: Union[str, Path]) -> Path:
    return p if isinstance(p, Path) else Path(p)


def find_records(path: str):
    search_path: str = f"{path}/**/signals/"
    all_paths = list(map(lambda x: str(to_path(x).parent),
                     glob.glob(search_path, recursive=True)))
    return all_paths


class RecordReader():
    def __init__(self, path: Union[str, Path]):
        self.path = to_path(path)

    def load_signal(self, sig_name):
        return xr.open_zarr(self.path / "signals" / sig_name / "dataset")

    def load_metadata(self):
        with open(self.path / "meta.json", "r") as meta:
            return json.load(meta)

    def load_crf_metadata(self):
        with open(self.path / "crf.json", "r") as meta:
            return json.load(meta)


In [3]:
records = find_records(("./"))
print(records, flush=True)


['Dataset1/MMC-HFrEF/KINO/Sub-EY/MMC-HFrEF_EY_kinoV1_20190107-153210_gwd5ar2K2tvvIo', 'Dataset1/MMC-HFrEF/KINO/Sub-MT/MMC-HFrEF_MT_kinoV1_20191210-161556_gwX8ymbEkuvvIo', 'Dataset1/MMC-HFrEF/KINO/Sub-WZ/MMC-HFrEF_WZ_kinoV1_20190930-145749_1WdAWm9mNfOGux', 'Dataset1/MMC-HFrEF/KINO/Sub-YI/MMC-HFrEF_YI_kinoV1_20190909-150010_gwkwwq1lvIvvIo', 'Dataset1/MMC-HFrEF/KINO/Sub-MV/MMC-HFrEF_MV_kinoV1_20190103-124526_gwdW4mA7nivvIo', 'Dataset1/MMC-HFrEF/KINO/Sub-RK/MMC-HFrEF_RK_kinoV1_20190506-142648_gwo4YOgXnIvvIo', 'Dataset1/MMC-HFrEF/KINO/Sub-YT/MMC-HFrEF_YT_kinoV1_20190603-145424_gwokar1wzTvvIo', 'Dataset1/MMC-HFrEF/KINO/Sub-UF/MMC-HFrEF_UF_kinoV1_20190624-145537_gwR5aOzoqIvvIo', 'Dataset1/MMC-HFrEF/KINO/Sub-PU/MMC-HFrEF_PU_kinoV1_20190211-154927_gwd4wQLYnsvvIo', 'Dataset1/MMC-HFrEF/KINO/Sub-QR/MMC-HFrEF_QR_kinoV1_20190812-145831_gwRaDplewcvvIo', 'Dataset1/MMC-HFrEF/KINO/Sub-WJ/MMC-HFrEF_WJ_kinoV1_20190520-145741_gwoa9Rdn8ivvIo', 'Dataset1/MMC-HFrEF/KINO/Sub-WF/MMC-HFrEF_WF_kinoV1_20190311-152

In [4]:
reader = RecordReader(records[0])


In [5]:
# Get Energy signals
nrg_lin_df = reader.load_signal("scg-k").nrg.sel(motion="lin").to_pandas()
nrg_rot_df = reader.load_signal("scg-k").nrg.sel(motion="rot").to_pandas()
pwr_lin_df = reader.load_signal("scg-k").pwr.sel(motion="lin").to_pandas()
pwr_rot_df = reader.load_signal("scg-k").pwr.sel(motion="rot").to_pandas()
resp = reader.load_signal("rsp").signal.to_pandas()


/home/brandon/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [6]:
len(records)


891

In [7]:
import pandas as pd

df_all = pd.DataFrame(columns=['age', 'sex', 'weight', 'weight_unit', 'height',
                      'height_unit', 'subject_id', 'study_id', 'hf_type', 'nrg_lin', 'nrg_rot'])

for r in records:
    reader = RecordReader(r)
    metadata = reader.load_metadata()
    crf_data = reader.load_crf_metadata()

    # Get Energy signals
    nrg_lin = reader.load_signal("scg-k").nrg.sel(motion="lin").to_pandas()
    nrg_rot = reader.load_signal("scg-k").nrg.sel(motion="rot").to_pandas()
    pwr_lin = reader.load_signal("scg-k").pwr.sel(motion="lin").to_pandas()
    pwr_rot = reader.load_signal("scg-k").pwr.sel(motion="rot").to_pandas()

    # Create a new row in the dataframe
    row = {
        'age': metadata['subject']['age']['value'],
        'sex': metadata['subject']['sex'],
        'weight': metadata['subject']['weight']['value'],
        'weight_unit': metadata['subject']['weight']['units'],
        'height': metadata['subject']['height']['value'],
        'height_unit': metadata['subject']['height']['units'],
        'subject_id': crf_data['subject_id'],
        'study_id': crf_data['study_id'],
        'hf_type': crf_data['hf_type'],
        'nrg_lin': nrg_lin,
        'nrg_rot': nrg_rot,
        'pwr_lin': pwr_lin,
        'pwr_rot': pwr_rot,
        'resp': resp,
    }
    df_all.loc[metadata['id']] = pd.Series(row)


In [8]:
df_signals = pd.DataFrame(
    columns=['nrg_lin', 'nrg_rot', 'pwr_lin', 'pwr_rot', 'resp'])
for r in records:
    reader = RecordReader(r)
    metadata = reader.load_metadata()
    crf_data = reader.load_crf_metadata()

    # Reading signals
    nrg_lin = reader.load_signal("scg-k").nrg.sel(motion="lin").to_pandas()
    nrg_rot = reader.load_signal("scg-k").nrg.sel(motion="rot").to_pandas()
    pwr_lin = reader.load_signal("scg-k").pwr.sel(motion="lin").to_pandas()
    pwr_rot = reader.load_signal("scg-k").pwr.sel(motion="rot").to_pandas()
    resp_df = reader.load_signal("rsp").signal.to_pandas()

    # Create a new row in the dataframe
    row = {

        'nrg_lin': nrg_lin,
        'nrg_rot': nrg_rot,
        'pwr_lin': pwr_lin,
        'pwr_rot': pwr_rot,
        'resp': resp_df

    }

    df_signals.loc[metadata['id']] = pd.Series(row)


In [61]:
df_all.head()


,age,sex,weight,weight_unit,height,height_unit,subject_id,study_id,hf_type,nrg_lin,nrg_rot
MMC-HFrEF_EY_kinoV1_20190107-153210_gwd5ar2K2tvvIo,70.0,Male,106,kg,1,m,EY,MMC-HFrEF,HFrEF,106,1
MMC-HFrEF_MT_kinoV1_20191210-161556_gwX8ymbEkuvvIo,70.0,Male,77,kg,1,m,MT,MMC-HFrEF,HFrEF,77,1
MMC-HFrEF_WZ_kinoV1_20190930-145749_1WdAWm9mNfOGux,72.0,Female,60,kg,1,m,WZ,MMC-HFrEF,HFrEF,60,1
MMC-HFrEF_YI_kinoV1_20190909-150010_gwkwwq1lvIvvIo,67.0,Male,78,kg,1,m,YI,MMC-HFrEF,HFrEF,78,1
MMC-HFrEF_MV_kinoV1_20190103-124526_gwdW4mA7nivvIo,68.0,Male,106,kg,1,m,MV,MMC-HFrEF,HFrEF,106,1


In [62]:
df_signals.head()


,nrg_lin,nrg_rot,pwr_lin,pwr_rot,resp
MMC-HFrEF_EY_kinoV1_20190107-153210_gwd5ar2K2tvvIo,time 0 8.143398e-41 5 2.974699e-...,time 0 3.429861e-19 5 2.323705e-...,time 0 -1.770610e-25 5 -1.521891e-...,time 0 -3.870084e-24 5 -6.804947e-...,time 0 0.341180 5 0.341047 10 ...
MMC-HFrEF_MT_kinoV1_20191210-161556_gwX8ymbEkuvvIo,time 0 4.827842e-41 5 3.143857e-...,time 0 3.722609e-19 5 1.496852e-...,time 0 -1.134743e-26 5 -2.053021e-...,time 0 -3.448829e-23 5 -1.442980e-...,time 0 0.062943 5 0.062854 10 ...
MMC-HFrEF_WZ_kinoV1_20190930-145749_1WdAWm9mNfOGux,time 0 1.008940e-39 5 4.981065e-...,time 0 1.613923e-19 5 7.237731e-...,time 0 2.180500e-25 5 2.554589e-...,time 0 1.942912e-23 5 -5.657292e-...,time 0 -0.237004 5 -0.236877 10 ...
MMC-HFrEF_YI_kinoV1_20190909-150010_gwkwwq1lvIvvIo,time 0 1.621356e-41 5 3.310214e-...,time 0 1.505611e-19 5 6.988511e-...,time 0 -4.326821e-26 5 -2.824252e-...,time 0 4.672307e-24 5 -3.064759e-...,time 0 -0.058702 5 -0.058662 10 ...
MMC-HFrEF_MV_kinoV1_20190103-124526_gwdW4mA7nivvIo,time 0 4.274874e-42 5 3.747583e-...,time 0 2.533075e-19 5 1.138278e-...,time 0 2.582036e-26 5 9.574861e-...,time 0 -3.296534e-23 5 1.806763e-...,time 0 0.251862 5 0.251774 10 ...


In [63]:
df_signals.shape


(891, 5)

In [64]:
df_all.shape


(891, 11)